In [ ]:
import sys
import os
import collections
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy import stats
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Final_College_Data.xlsx', index_col=0) 
data.head() 

In [ ]:
data.shape


In [ ]:
data = data[data['Decision'] =='Accepted']

data.shape

In [ ]:
data.describe()

In [ ]:
y = data.univName                 
x = data.drop('univName',axis=1) 

In [ ]:
# Label the University names to perform SMOTE
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)


In [ ]:
x1 = pd.get_dummies(x)

In [ ]:
# Perform over_sampling to balance the dataset
smote = SMOTE(sampling_strategy='not majority')
X1, Y1 = smote.fit_sample(x1,encoded_Y)


In [ ]:
# Scale the values
sc = RobustScaler()       # Robust scaler takes care of outliers as well
X = sc.fit_transform(X1)

In [ ]:
# One-hot encoding of the University Names
Y = np_utils.to_categorical(Y1)

In [ ]:
# Make the train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42,shuffle=True)


In [ ]:
#Multi-class Classifier Model
classifier = Sequential()
classifier.add(Dense(400, activation='relu', kernel_initializer='random_normal', input_dim=X_test.shape[1]))
classifier.add(Dense(800, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(100, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(36, activation='softmax', kernel_initializer='random_normal'))
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

classifier.fit(X_train,Y_train,batch_size=20,epochs=200,verbose=0)
eval_model = classifier.evaluate(X_train,Y_train)
print("Accuracy: ",eval_model[1]) 

In [ ]:
# Get the predicted class for each test sample

y_pred = classifier.predict_classes(X_test)
print(y_pred)


In [ ]:
# Generate confusion matrix to see the performance of classifier in classifying correctly

cm = confusion_matrix(Y_test.argmax(axis=1),y_pred)
ax = plt.subplot()
sns.heatmap(cm,annot=False,ax=ax);
ax.set_xlabel('Predicted');
ax.set_ylabel('Actual');
ax.set_title('Confusion Matrix');
plt.show()


## Random Forest Nearest Neighbour classifier and SVM Classifier ##


In [ ]:
def Nearest_Neighbour_classifier(train_input_data,train_output_data,test_input_data,test_output_data):
    neighbour_list = []
    accuracy_percent = []
    for neighbours in range(1,101,5):
        clf = neighbors.KNeighborsClassifier(neighbours, weights='uniform')
        clf.fit(train_input_data, train_output_data)
        predicted_output = clf.predict(test_input_data)
        if isinstance(predicted_output,list) ==False:
            predicted_output = predicted_output.tolist()
        if isinstance(test_output_data,list) ==False:
            test_output_data = test_output_data.tolist()
        error_list = []
        for i in range(len(test_output_data)):
            cur_univ_similarities =  similar_univs[similar_univs['univName'] == predicted_output[i]]
            cur_univ_similarity_list = cur_univ_similarities.values.tolist()
            cur_univ_similarity_list = [item for sublist in cur_univ_similarity_list for item in sublist]
            if test_output_data[i] in cur_univ_similarity_list[1:]:
                error_list.append(0)
            else:
                error_list.append(1)
        neighbour_list.append(neighbours)
        accuracy_percent.append(100 -((sum(error_list)/float(len(error_list))) * 100))
    neighbour_list = np.array(neighbour_list)
    accuracy_percent = np.array(accuracy_percent)
    plt.plot(neighbour_list,accuracy_percent)
    plt.xlabel('Number of nearest neighbors')
    plt.ylabel('Percent of accuracy')
    plt.title('Varation of accuracy with nearest neighbours')
    plt.grid(True)
    plt.savefig("knn1.png")
    plt.show()
    return predicted_output

In [ ]:
def SVM_classifier(train_input_data,train_output_data,test_input_data,test_output_data):
    clf = svm.SVC()
    clf.fit(train_input_data,train_output_data)
    predicted_output = clf.predict(test_input_data)
    error_list = []
    predicted_output = predicted_output.tolist()
    test_output_data  = test_output_data.tolist()
    for i in range(len(test_output_data)):
        cur_univ_similarities =  similar_univs[similar_univs['univName'] == predicted_output[i]]
        cur_univ_similarity_list = cur_univ_similarities.values.tolist()
        cur_univ_similarity_list = [item for sublist in cur_univ_similarity_list for item in sublist]
        if test_output_data[i] in cur_univ_similarity_list[1:]:
            error_list.append(0)
        else:
            error_list.append(1)
    return predicted_output


In [ ]:
data = pandas.read_csv('processed_data.csv')
data = data.drop('Unnamed: 0',1)
data = data.drop('Unnamed: 0.1',1)

In [ ]:
similar_univs = pandas.read_csv('similar_universities.csv')
random_indices = permutation(data.index)
test_cutoff = math.floor(len(data)/5)
test = data.loc[random_indices[1:test_cutoff]]
train = data.loc[random_indices[test_cutoff:]]
train_output_data = train['univName']
train_input_data = train
train_input_data = train_input_data.drop('univName',1)
test_output_data = test['univName']
test_input_data = test
test_input_data = test_input_data.drop('univName',1)

In [ ]:
output = Random_Forest_classifier(train_input_data,train_output_data,test_input_data,test_output_data)
output = SVM_classifier(train_input_data,train_output_data,test_input_data,test_output_data)

## Collaborative Filtering Based Recommender System ##

In [ ]:
# import TfidfVector from sklearn.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
def create_similarity_matrix(new_description, overall_descriptions):
#Append the new description to the overall set.
      overall_descriptions.append(new_description)
        # Define a tfidf vectorizer and remove all stopwords.
      tfidf = TfidfVectorizer(stop_words="english")
        #Convert tfidf matrix by fitting and transforming the data.
      tfidf_matrix = tfidf.fit_transform(overall_descriptions)
        # output the shape of the matrix.
      tfidf_matrix.shape
        # calculating the cosine similarity matrix.
      cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)
      return cosine_sim

In [ ]:
def get_recommendations(new_description,overall_descriptions):
# create the similarity matrix
  cosine_sim = create_similarity_matrix(new_description,overall_descriptions)
# Get pairwise similarity scores of all the students with new student.
  sim_scores = list(enumerate(cosine_sim[-1]))
# Sort the descriptions based on similarity score.
  sim_scores = sorted(sim_scores,key =lambda x:x[1],reverse= True )
# Get the scores of top 10 descriptions.
  sim_scores = sim_scores[1:10]
# Get the student indices.
  indices = [i[0]for i in sim_scores]
  return data.iloc[indices]

In [ ]:
new_description = pd.Series('physics science 78')
get_recommendations(new_description,descriptions)